In [1]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten

import cv2

import pandas as pd
import numpy as np
import matplotlib.image as mpimg

from helpers import BirdsEye

Using TensorFlow backend.
/Users/mithi/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Model Parameters 
BATCH_SIZE = 256
NUMBER_OF_EPOCHS = 20
VALIDATION_SIZE = 0.10
OPTIMIZER_TYPE = "adam"
LOSS_TYPE = "mse"
MODELH5_NAME = "./misc/modelv2.h5"
MODELJSON_NAME = "./misc/modelv2.json"
INPUT_SHAPE = (66, 200, 3)

# For warping the image
SOURCE_POINTS = [(0, 43), (130, 15), (190, 15), (320, 43)]
DEST_POINTS = [(130, 50), (130, 10), (190, 10), (190, 50)]
BIRDS_EYE = BirdsEye(SOURCE_POINTS, DEST_POINTS)

# For cropping the image
YSTART, YSTOP = 100, 166
XSTART, XSTOP = 50, 250

IMAGE_DIR = "./samples/v2-images/"
CSV_FILE = "./samples/logs-v2.csv"
DATA = pd.read_csv(CSV_FILE)

In [ ]:
# IMPORTANT: Don't run if there's nothing wrong in your csv files
# Delete rows in logs that do not have existing image file in directory (IE corrupted log file)

OLD_CSV_FILE = "./samples/logs-v2-original.csv"

DATA = pd.read_csv(OLD_CSV_FILE)
print("number of rows:", len(DATA))
print(DATA.tail())

for i in range(0, len(DATA)):
    image_name = DATA['NAME'][i]
    image_path = IMAGE_DIR + image_name
    try:
        unprocessed_img = mpimg.imread(image_path)
        print(image_name, "exists.")
    except:
        print("delete:", i, "name:", image_name)
        DATA = DATA.drop([i])

print("Cleaned data:", len(DATA))

NEW_CSV_FILE = "./samples/logs-v2.csv"
DATA.to_csv(NEW_CSV_FILE, index=False, encoding='utf8')

DATA = pd.read_csv(NEW_CSV_FILE)
print("number of rows:", len(DATA))

In [3]:
def random_flip(image, steer):
    if np.random.random() > 0.5:
        return np.fliplr(image), -steer
    else:
        return image, steer

In [4]:
def get_processed_data(x, data):
    
    i = data.index[x]
    steer =  data['STEER'][i]
    image_path = IMAGE_DIR + data['NAME'][i]
    unprocessed_img = mpimg.imread(image_path)
    cropped_img = unprocessed_img[YSTART:YSTOP, :, :]
    sky_img = BIRDS_EYE.skyview(cropped_img)
    img = sky_img[:, XSTART:XSTOP]
    cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    
    return random_flip(img, steer)

In [5]:
def generate_samples(data, batch_size):

    while True:

        SIZE = len(data)
        data.sample(frac=1) #shuffle

        for start in range(0, SIZE, batch_size):
            images, steers = [], []

            for i in range(start, start + batch_size):
                if i < SIZE:
                    image, steer = get_processed_data(i, data)
                    steers.append(steer)
                    images.append(image)

            yield (np.array(images), np.array(steers))

In [6]:
def build_modified_nvidia_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
    model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    model.compile(optimizer = OPTIMIZER_TYPE, loss = LOSS_TYPE)
    return model

In [7]:
model = build_modified_nvidia_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 66, 200, 3)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 31, 98, 24)    1824        lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 14, 47, 36)    21636       convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 5, 22, 48)     43248       convolution2d_2[0][0]            
___________________________________________________________________________________________

In [8]:
print('Training model...')

TRAINING_DATA, VALIDATION_DATA = train_test_split(DATA, test_size=VALIDATION_SIZE)
TOTAL_TRAIN_DATA = len(TRAINING_DATA)
TOTAL_VALID_DATA = len(VALIDATION_DATA)

training_generator = generate_samples(TRAINING_DATA, batch_size=BATCH_SIZE)
validation_generator = generate_samples(VALIDATION_DATA, batch_size=BATCH_SIZE)

model.fit_generator(training_generator,
    samples_per_epoch=TOTAL_TRAIN_DATA,
    validation_data=validation_generator,
    nb_val_samples=TOTAL_VALID_DATA,
    nb_epoch=NUMBER_OF_EPOCHS,
    verbose=1)

print('...Model trained.')

Training model...
Epoch 1/20
21028/21028 [==============================] - 401s - loss: 80.0627 - val_loss: 72.3311
Epoch 2/20
21028/21028 [==============================] - 305s - loss: 71.2827 - val_loss: 70.5750
Epoch 3/20
21028/21028 [==============================] - 462s - loss: 69.7131 - val_loss: 70.6674
Epoch 4/20
21028/21028 [==============================] - 466s - loss: 68.7946 - val_loss: 69.9596
Epoch 5/20
21028/21028 [==============================] - 387s - loss: 68.7463 - val_loss: 70.4908
Epoch 6/20
21028/21028 [==============================] - 205s - loss: 68.3200 - val_loss: 66.9899
Epoch 7/20
21028/21028 [==============================] - 205s - loss: 67.6626 - val_loss: 67.6986
Epoch 8/20
21028/21028 [==============================] - 206s - loss: 67.5285 - val_loss: 67.0844
Epoch 9/20
21028/21028 [==============================] - 205s - loss: 67.1530 - val_loss: 67.4116
Epoch 10/20
21028/21028 [==============================] - 210s - loss: 66.7750 - val_loss:

In [9]:
print('Saving model...')

model.save(MODELH5_NAME)

with open(MODELJSON_NAME, "w") as json_file:
    json_file.write(model.to_json())

print("...Model Saved.")

Saving model...
...Model Saved.
